# Setup

In [ ]:
# OpenSlide
!apt update && apt install -y openslide-tools
!pip install openslide-python

In [ ]:
# ASAP
!wget https://github.com/computationalpathologygroup/ASAP/releases/download/ASAP-2.1/ASAP-2.1-py38-Ubuntu2004.deb
!apt install libdcmtk14
!apt install libopenslide0
!ls
!dpkg -i ASAP-2.1-py38-Ubuntu2004.deb

# add ASAP to pythonpath
import sys
asap_path = "/opt/ASAP/bin"
if asap_path not in sys.path:
    sys.path.append(asap_path);
print(sys.path);

# Mask Generation

In [ ]:
import openslide
import multiresolutionimageinterface as mir
import os

TUMOR_PATH = '/content/drive/MyDrive/Thesis/Tumor/'
TUMOR_ANN_PATH = '/content/drive/MyDrive/Thesis/Tumor_Annotations/'
TUMOR_MASK_PATH = '/content/drive/MyDrive/Thesis/Tumor_Masks/'

def ann_to_mask(wsi):
  mr_image = reader.open(TUMOR_PATH + wsi)
  ann_path = wsi.split('.', 1)[0] + '.xml'

  # load annotation
  xml_repository.setSource(TUMOR_ANN_PATH + ann_path)
  xml_repository.load()

  # save mask file (warning: takes long)
  mask_path = wsi.replace('tumor', 'mask')
  annotation_mask.convert(annotation_list, TUMOR_MASK_PATH + mask_path, 
                          mr_image.getDimensions(), mr_image.getSpacing(), 
                          label_map, conversion_order)
  
if __name__ == '__main__':
  reader = mir.MultiResolutionImageReader()

  annotation_list = mir.AnnotationList()
  xml_repository = mir.XmlRepository(annotation_list)
  annotation_mask = mir.AnnotationToMask()

  # adjust labels based on dataset (camelyon16 vs 17)
  camelyon17_type_mask = False
  label_map = {'metastases': 1, 'normal': 2} if camelyon17_type_mask else {'_0': 255, '_1': 255, '_2': 0}
  conversion_order = ['metastases', 'normal'] if camelyon17_type_mask else  ['_0', '_1', '_2']

  # get list of all tumor WSIs and convert to masks
  tumor_wsis = os.listdir(TUMOR_PATH)
  for wsi in tumor_wsis:
    ann_to_mask(wsi)